# Basic Text Processing with Spacy
    
As a consultant for [DelFalco's Italian Restaurant](https://defalcosdeli.com/index.html).
The owner wants to identify whether there are any foods on their menu that diners find disappointing. 

<img src="https://i.imgur.com/8DZunAQ.jpg" alt="Meatball Sub" width="250"/>

The business owner suggested to use diner reviews from the Yelp website to determine which dishes people liked and disliked. having pulled the data from Yelp. Before we get to analysis, let's have a quick look at the data you have to work with.


In [ ]:
import pandas as pd

# Load in the data from JSON file
data = pd.read_json('../nlp/restaurant.json')
data.head()

The owner also provided this list of menu items and common alternate spellings.

In [ ]:
menu = ["Cheese Steak", "Cheesesteak", "Steak and Cheese", "Italian Combo", "Tiramisu", "Cannoli",
        "Chicken Salad", "Chicken Spinach Salad", "Meatball", "Pizza", "Pizzas", "Spaghetti",
        "Bruchetta", "Eggplant", "Italian Beef", "Purista", "Pasta", "Calzones",  "Calzone",
        "Italian Sausage", "Chicken Cutlet", "Chicken Parm", "Chicken Parmesan", "Gnocchi",
        "Chicken Pesto", "Turkey Sandwich", "Turkey Breast", "Ziti", "Portobello", "Reuben",
        "Mozzarella Caprese",  "Corned Beef", "Garlic Bread", "Pastrami", "Roast Beef",
        "Tuna Salad", "Lasagna", "Artichoke Salad", "Fettuccini Alfredo", "Chicken Parmigiana",
        "Grilled Veggie", "Grilled Veggies", "Grilled Vegetable", "Mac and Cheese", "Macaroni",  
         "Prosciutto", "Salami"]

# Step 1: Plan the Analysis

Given the data from Yelp and the list of menu items, to find how a menu disappointed a customer, we're going to use a SpaCy's smart matcher

# Step 2: Find items in one review

We'll pursue this plan of calculating average scores of the reviews mentioning each menu item.

As a first step, we'll write code to extract the foods mentioned in a single review.

Since menu items are multiple tokens long, we'll use `PhraseMatcher` which can match series of tokens.

In [ ]:
import spacy
from spacy.matcher import PhraseMatcher

index_of_review_to_test_on = 14
text_to_test_on = data.text.iloc[index_of_review_to_test_on]

# Load the SpaCy model
nlp = spacy.blank('en')

# Create the tokenized version of text_to_test_on
review_doc = nlp(text_to_test_on)

# Create the PhraseMatcher object. The tokenizer is the first argument. Use attr = 'LOWER' to make consistent capitalization
matcher = PhraseMatcher(nlp.vocab, attr='LOWER')

# Create a list of tokens for each item in the menu
menu_tokens_list = [nlp(item) for item in menu]

# Add the item patterns to the matcher.

matcher.add("MENU", menu_tokens_list)

# Find matches in the review_doc
matches = matcher(review_doc)

for match in matches:
   print(f"Token number {match[1]}: {review_doc[match[1]:match[2]]}")


# Step 3: Matching on the whole dataset

Now run this matcher over the whole dataset and collect ratings for each menu item. Each review has a rating, `review.stars`. For each item that appears in the review text (`review.text`), append the review's rating to a list of ratings for that item. The lists are kept in a dictionary `item_ratings`.

To get the matched phrases, we reference the `PhraseMatcher` documentation for the structure of each match object:

>A list of `(match_id, start, end)` tuples, describing the matches. A match tuple describes a span `doc[start:end]`. The `match_id` is the ID of the added match pattern.

In [ ]:
from collections import defaultdict

# item_ratings is a dictionary of lists. If a key doesn't exist in item_ratings,
# the key is added with an empty list as the value.
item_ratings = defaultdict(list)

for idx, review in data.iterrows():
    doc = nlp(review.text)
    # Using the matcher from the previous exercise
    matches = matcher(doc)
    
    # Create a set of the items found in the review text
    found_items = set([doc[match[1]:match[2]].lower_ for match in matches])
    
    # Update item_ratings with rating for each item in found_items
    # Transform the item strings to lowercase to make it case insensitive
    for item in found_items:
        item_ratings[item].append(review.stars)


# Step 4: What's the worst reviewed item?

Using these item ratings, let's find the menu item with the worst average rating.

In [ ]:
# Calculate the mean ratings for each menu item as a dictionary
mean_ratings = {item:sum(rating)/len(rating) for item,rating in item_ratings.items()}
print(mean_ratings)
# worst_item = sorted(mean_ratings,key = lambda pair: mean_ratings.get(pair))
worst_item = sorted(mean_ratings,key = lambda key: mean_ratings[key])[0]
# worst_item = sorted(mean_ratings,key = mean_ratings.get)[0:5]
print(worst_item)

In [ ]:
# out the worst item, along with its average rating. 

print(worst_item)
print(mean_ratings[worst_item])

# Step 5: Are counts important here?

Similar to the mean ratings, we can calculate the number of reviews for each item.

In [ ]:
counts = {item: len(ratings) for item, ratings in item_ratings.items()}

item_counts = sorted(counts, key=counts.get, reverse=True)
for item in item_counts:
    print(f"{item:>25}{counts[item]:>5}")

Here is code to print the 10 best and 10 worst rated items. Look at the results, and let the manager decide whether it's important to consider the number of reviews when interpreting scores of which items are best and worst.

In [ ]:
sorted_ratings = sorted(mean_ratings, key=mean_ratings.get)

print("Worst rated menu items:")
for item in sorted_ratings[:10]:
    print(f"{item:20} Ave rating: {mean_ratings[item]:.2f} \tcount: {counts[item]}")
    
print("\n\nBest rated menu items:")
for item in sorted_ratings[-10:]:
    print(f"{item:20} Ave rating: {mean_ratings[item]:.2f} \tcount: {counts[item]}")